# Streaming

> SSE streaming utilities and helpers

In [ ]:
#| default_exp core.streaming

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import asyncio
import json
from typing import AsyncGenerator, Callable, Optional, Any, Dict, List, Union
from datetime import datetime
from dataclasses import dataclass
from fasthtml.common import EventStream, sse_message, Div

## Core Classes

In [ ]:
#| export
@dataclass
class StreamConfig:
    """Configuration for SSE streaming"""
    heartbeat_interval: float = 30.0
    timeout: Optional[float] = None
    send_initial_message: bool = True
    initial_message: str = "Connected"
    send_close_message: bool = True
    close_message: str = "Connection closed"
    debug: bool = False

In [ ]:
#| export
class SSEStream:
    """Generic SSE stream handler"""
    
    def __init__(
        self,
        config: Optional[StreamConfig] = None  # TODO: Add description
    ):
        """Initialize the SSE stream.
        
        Args:
            config: Stream configuration
        """
        self.config = config or StreamConfig()
        self._active = False
        self._message_count = 0
    
    async def stream(self,
                    data_source: Union[AsyncGenerator, Callable],
                    transform_fn: Optional[Callable] = None) -> AsyncGenerator[str, None]:
        """Stream data from a source through SSE.
        
        Args:
            data_source: Async generator or callable that produces data
            transform_fn: Optional function to transform data before sending
            
        Yields:
            SSE formatted strings
        """
        self._active = True
        
        try:
            # Send initial message
            if self.config.send_initial_message:
                yield f": {self.config.initial_message}\n\n"
            
            # Create async generator from callable if needed
            if callable(data_source) and not hasattr(data_source, '__anext__'):
                data_source = data_source()
            
            # Main streaming loop
            while self._active:
                try:
                    # Get data with heartbeat timeout
                    data = await asyncio.wait_for(
                        data_source.__anext__(),
                        timeout=self.config.heartbeat_interval
                    )
                    
                    # Transform data if function provided
                    if transform_fn:
                        data = await transform_fn(data) if asyncio.iscoroutinefunction(transform_fn) else transform_fn(data)
                    
                    # Send data
                    if data is not None:
                        self._message_count += 1
                        yield self._format_message(data)
                    
                except asyncio.TimeoutError:
                    # Send heartbeat
                    yield f": heartbeat {datetime.now().isoformat()}\n\n"
                    
                except StopAsyncIteration:
                    # Data source exhausted
                    break
                    
        except Exception as e:
            if self.config.debug:
                yield f": error {str(e)}\n\n"
            raise
            
        finally:
            self._active = False
            if self.config.send_close_message:
                yield f": {self.config.close_message}\n\n"
    
    def _format_message(
        self,
        data: Any  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Format data as SSE message.
        
        Args:
            data: Data to format
            
        Returns:
            SSE formatted string
        """
        if isinstance(data, str):
            return f"data: {data}\n\n"
        elif isinstance(data, dict):
            return sse_message(data)
        elif hasattr(data, '__html__'):
            # FastHTML element
            return sse_message(data)
        else:
            # Try to convert to JSON
            try:
                return f"data: {json.dumps(data)}\n\n"
            except (TypeError, ValueError):
                return f"data: {str(data)}\n\n"
    
    def stop(self):
        """Stop the stream."""
        self._active = False

In [ ]:
#| export
class OOBStreamBuilder:
    """Build SSE messages with OOB (Out-of-Band) swaps"""
    
    def __init__(self):
        """Initialize the OOB stream builder."""
        self.elements: List[Any] = []
    
    def add_element(self,
                   element: Any,
                   target_id: Optional[str] = None,
                   swap_mode: str = "innerHTML",
                   wrap: bool = True) -> 'OOBStreamBuilder':
        """Add an element with OOB swap configuration.
        
        Args:
            element: The element to add
            target_id: Target element ID for OOB swap
            swap_mode: Swap mode (innerHTML, outerHTML, beforeend, afterbegin, etc.)
            wrap: If True and target_id is provided, wrap content in a Div with OOB attributes.
                  If False, add OOB attributes directly to the element.
            
        Returns:
            Self for chaining
        """
        if target_id:
            if wrap and swap_mode == "innerHTML":
                # For innerHTML swaps, wrap the content in a container with the target ID
                # This is the most common case for replacing content
                wrapper = Div(element, id=target_id, hx_swap_oob="innerHTML")
                self.elements.append(wrapper)
            elif wrap and swap_mode != "outerHTML":
                # For other swap modes (beforeend, afterbegin, etc.), also wrap
                wrapper = Div(element, id=target_id, hx_swap_oob=swap_mode)
                self.elements.append(wrapper)
            else:
                # For outerHTML swaps or when wrap=False, add attributes directly to element
                if hasattr(element, 'attrs'):
                    # Only set ID if element doesn't already have one or if it matches target_id
                    if not element.attrs.get('id') or element.attrs.get('id') == target_id:
                        element.attrs['id'] = target_id
                    element.attrs['hx_swap_oob'] = swap_mode if swap_mode != "innerHTML" else "true"
                elif isinstance(element, dict):
                    if not element.get('id') or element.get('id') == target_id:
                        element['id'] = target_id
                    element['hx_swap_oob'] = swap_mode if swap_mode != "innerHTML" else "true"
                self.elements.append(element)
        else:
            # No target_id, just add the element as-is
            self.elements.append(element)
        
        return self
    
    def add_elements(self, elements: List[tuple]) -> 'OOBStreamBuilder':
        """Add multiple elements with OOB configurations.
        
        Args:
            elements: List of tuples: (element, target_id, swap_mode, wrap) or
                     (element, target_id, swap_mode) or (element, target_id) or (element,)
            
        Returns:
            Self for chaining
        """
        for item in elements:
            if len(item) == 4:
                element, target_id, swap_mode, wrap = item
            elif len(item) == 3:
                element, target_id, swap_mode = item
                wrap = True  # Default to wrapping
            elif len(item) == 2:
                element, target_id = item
                swap_mode = "innerHTML"
                wrap = True
            else:
                element = item[0] if isinstance(item, tuple) else item
                target_id = None
                swap_mode = "innerHTML"
                wrap = True
            
            self.add_element(element, target_id, swap_mode, wrap)
        
        return self
    
    def build(self) -> str:
        """Build the SSE message with all elements.
        
        Returns:
            SSE formatted message
        """
        if not self.elements:
            return ""
        
        if len(self.elements) == 1:
            return sse_message(self.elements[0])
        
        # Wrap multiple elements in a container
        # container = Div(*self.elements)
        # return sse_message(container)
        return Div(*self.elements)
    
    def clear(self) -> 'OOBStreamBuilder':
        """Clear all elements.
        
        Returns:
            Self for chaining
        """
        self.elements = []
        return self

## Helper Functions

In [ ]:
#| export
async def sse_generator(data_source: Union[AsyncGenerator, List, Callable],
                       interval: float = 0.5,  # Interval between items for list sources
                       heartbeat: float = 30.0,  # Heartbeat interval
                       transform: Optional[Callable] = None) -> AsyncGenerator[str, None]:
    "Create an SSE generator from various data sources."
    # Handle different data source types
    if isinstance(data_source, list):
        # Convert list to async generator
        async def list_generator():
            "TODO: Add function description"
            for item in data_source:
                yield item
                await asyncio.sleep(interval)
        data_source = list_generator()
    elif callable(data_source) and not hasattr(data_source, '__anext__'):
        # Call the function to get the generator
        data_source = data_source()
    
    # Stream the data
    stream = SSEStream(StreamConfig(heartbeat_interval=heartbeat))
    async for message in stream.stream(data_source, transform):
        yield message

In [ ]:
#| export
def create_sse_endpoint(stream_fn: Callable,
                       content_type: str = "text/event-stream") -> Callable:
    "Create an SSE endpoint from a streaming function."
    async def endpoint(*args, **kwargs):
        "TODO: Add function description"
        return EventStream(stream_fn(*args, **kwargs))
    
    return endpoint

In [ ]:
#| export
async def stream_updates(source_queue: asyncio.Queue,
                        transform_fn: Optional[Callable] = None,  # Optional transformation function
                        config: Optional[StreamConfig] = None) -> AsyncGenerator[str, None]:
    "Stream updates from an async queue."
    config = config or StreamConfig()
    
    async def queue_generator():
        "TODO: Add function description"
        while True:
            message = await source_queue.get()
            yield message
    
    stream = SSEStream(config)
    async for message in stream.stream(queue_generator(), transform_fn):
        yield message

In [ ]:
#| export
def create_throttled_stream(source: AsyncGenerator,
                           min_interval: float = 0.1,  # Minimum interval between messages
                           max_buffer: int = 10) -> AsyncGenerator:
    "Create a throttled stream to prevent overwhelming clients."
    async def throttled():
        "TODO: Add function description"
        last_sent = 0
        buffer = []
        
        async for item in source:
            current_time = asyncio.get_event_loop().time()
            
            if current_time - last_sent >= min_interval:
                # Send immediately
                yield item
                last_sent = current_time
                
                # Flush buffer if any
                if buffer:
                    for buffered in buffer[:max_buffer]:
                        yield buffered
                        await asyncio.sleep(min_interval)
                    buffer.clear()
            else:
                # Buffer the message
                buffer.append(item)
                if len(buffer) > max_buffer:
                    buffer.pop(0)
        
        # Flush remaining buffer
        for buffered in buffer:
            yield buffered
            await asyncio.sleep(min_interval)
    
    return throttled()

## Examples

### Basic Streaming

In [ ]:
# Example: Basic SSE streaming
async def simple_counter():
    """Generate a simple counter stream."""
    for i in range(10):
        yield {"count": i}
        await asyncio.sleep(1)

# Create SSE stream
stream = SSEStream()

# Use in an endpoint
async def counter_endpoint():
    return EventStream(stream.stream(simple_counter()))

### OOB Updates

In [ ]:
# Example: Building OOB updates
from fasthtml.common import Div, Span, P

def build_status_update(status: str, count: int):
    """Build an OOB update for status and counter."""
    builder = OOBStreamBuilder()
    
    # Add status update
    builder.add_element(
        Span(status),
        target_id="status-display",
        swap_mode="innerHTML"
    )
    
    # Add counter update
    builder.add_element(
        Span(str(count)),
        target_id="counter",
        swap_mode="innerHTML"
    )
    
    return builder.build()

# Use in a stream
async def status_stream():
    for i in range(10):
        yield build_status_update(f"Processing {i}", i)
        await asyncio.sleep(1)

### Complete Integration

In [ ]:
# Example: Complete FastHTML integration
from fasthtml.common import fast_app, Div, H1

def create_streaming_app():
    app, rt = fast_app()
    
    # Queue for updates
    update_queue = asyncio.Queue()
    
    @rt("/stream")
    async def stream_endpoint():
        """SSE streaming endpoint."""
        return EventStream(stream_updates(update_queue))
    
    @rt("/")
    def index():
        return Div(
            H1("SSE Streaming Demo"),
            Div(
                id="updates",
                hx_ext="sse",
                sse_connect="/stream",
                sse_swap="message"
            ),
            Div(id="status-display"),
            Div(id="counter")
        )
    
    @rt("/send")
    async def send_update(message: str):
        """Send an update to all connected clients."""
        await update_queue.put({"message": message})
        return "Update sent"
    
    return app

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()